## Saurabh Dadhich 
### Indian Institute Of Technology, Roorkee

##  FaceMask Detection:

### Importing required packages: 

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import time
import cv2
import os

### Function returning location of face and predictions: 

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    
    #cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)  
    #-- return blob after mean subtraction, normalizing, and channel swapping.
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
   # print("Blob shape: ",blob.shape)             # (1, 3, 224, 224)
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
   # print("FaceNet output shape: ",detections.shape)
    
    faces = []
    locs = []
    preds = []

    for i in range(detections.shape[2]):
        # Probability associated with detection
        confidence = detections[0, 0, i, 2]
        
        # detections[0,0,i,0] = 1 for face not found     -- Box not defined 
        # detection[0,0,i,1]  = 1 for face found  
        
        # Defining threshold to be 0.5
        if confidence>0.5:
            # Computing box:
          #  print("detections place:  ",detections[0,0,i]) 
            
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            

            # extract the face, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB) 
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)  
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
            
    if len(faces) > 0:
            # for faster inference we'll make batch predictions on *all*
            # faces at the same time rather than one-by-one predictions 
            
            faces = np.array(faces, dtype="float32")
            preds = maskNet.predict(faces, batch_size=32)

    
    
    # return a 2-tuple of the face locations and their corresponding locations
    return (locs, preds)         

### Loading face detector model: 

In [3]:
prototxtPath = r"F:\Spark foundation\FaceMask Detector\face_detector_model\deploy.prototxt"
weightsPath = r"F:\Spark foundation\FaceMask Detector\face_detector_model\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

### Loading mask detection model: 

In [4]:
maskNet = load_model(r"F:\Spark foundation\FaceMask Detector\Mask_detector_model")

### Video capture from Web camera: 

In [7]:
cap = cv2.VideoCapture(0)

In [8]:
while True:
    # Capturing frame:
    ret, frame = cap.read()
    
    # imutils.resize() only tries to evaluate either the width or height argument,
    # and tries to preserve the original aspect ratio of the image,
    # while cv2.resize() gives you a free choice for height and width.
    frame = imutils.resize(frame, width=400)

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1)

    # if the `ESC` key was pressed, break from the loop
    if key == 27:    
        break
  
cv2.destroyAllWindows()

detections place:   [0.         1.         0.70062953 0.36716807 0.46710122 0.6014607
 0.8476614 ]
detections place:   [0.         1.         0.947679   0.37929457 0.49874926 0.6240377
 0.8921641 ]
detections place:   [0.        1.        0.8998814 0.383971  0.5074719 0.6237159 0.8902409]
detections place:   [0.         1.         0.7695738  0.38054544 0.49221265 0.6204197
 0.90582335]
detections place:   [0.         1.         0.7695738  0.38054544 0.49221265 0.6204197
 0.90582335]
detections place:   [0.         1.         0.7982419  0.38807473 0.51010066 0.6293512
 0.9008605 ]
detections place:   [0.         1.         0.75132936 0.40283793 0.5026115  0.6340361
 0.89364266]
detections place:   [0.         1.         0.5830169  0.40524003 0.5117115  0.63275516
 0.9033816 ]
detections place:   [0.         1.         0.5560817  0.41433316 0.5075011  0.6407538
 0.90536547]
detections place:   [0.         1.         0.7522373  0.4016484  0.5159924  0.63093257
 0.9029213 ]
detections plac

#### END